In [1]:
from typing import List

import torch
import torch.nn.functional as F

from numpy import random
import bz2

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

Network

In [2]:
model_name = "meta-llama/Llama-3.2-1B"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quant_config,
    # device_map=torch.device("cuda"),
    device_map="auto"
)
llm.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

Configuration

In [ ]:
class LLMCompression:

    def __init__(self,
        llm_name: str,
        context_size: int, # context_window: int,
    ):
        self.llm_name = llm_name
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for better performance
            bnb_4bit_use_double_quant=True,  # Double quantization for memory efficiency
        )
        self.llm = AutoModelForCausalLM.from_pretrained(
            llm_name,
            quantization_config=quant_config,
            # device_map=torch.device("cuda"),
            device_map="auto"
        )
        self.llm.eval()
        self.tokenizer = AutoTokenizer.from_pretrained(llm_name)

        self.context_size = context_size
    
    def _pad(self, tokens):
        if tokens.shape[0] % self.context_size == 0:
            return tokens, torch.zeros(tokens.shape[0], device=tokens.device)
        pad_len = self.context_size - tokens.shape[0] % self.context_size

        pads = torch.full([pad_len], self.tokenizer.eos_token_id, device=tokens.device)
        padded_tokens = torch.cat([tokens, pads])

        return padded_tokens, pad_len
    
    # rank from "stable" sort
    def _get_rank(self, logits, token_ids):  # (B, N, V) (B, N)
        # count the strictly the number of greater values
        selected_logits = logits.gather(-1, token_ids[..., None]).squeeze(-1)
        n_gt = (logits > selected_logits[..., None]).sum(-1)  # (B, N)

        # "mimic" stable sorting
        eq = (logits == selected_logits[..., None])  # (B, N, V)
        mask = torch.arange(logits.shape[1], device=logits.device).unsqueeze(0) < token_ids
        print("mask", mask.shape, token_ids.shape)
        n_eq = (eq*mask.unsqueeze(-1)).sum(-1)
        print(n_eq.shape)

        return n_gt + n_eq
    
    def argsort_solution(self, logits, targets):
        sort = torch.argsort(-logits, -1)
        return torch.where(sort == targets[..., None])[1]

    @torch.no_grad
    def encode(self, s):
        tokens = self.tokenizer(s, return_tensors="pt")
        tokens = tokens["input_ids"].squeeze()
        tokens = tokens.to(self.llm.device)

        tokens, pad_len = self._pad(tokens[1:])
        tokens = tokens.view(-1, self.context_size)

        bos = torch.full([tokens.shape[0]], self.tokenizer.bos_token_id, device=tokens.device).unsqueeze(1)
        tokens = torch.cat((bos, tokens), 1)

        # ranks = torch.empty_like(tokens[:, :-1])
        # for idx in range(self.context_size):
        #     next_tokens = self.llm(tokens[:, :idx+1])
            
        #     rank = self.argsort_solution(next_tokens.logits[:, -1, :], tokens[:, idx+1])
        #     ranks[:, idx] = rank

        output = self.llm(tokens[:, :-1])
        ranks = self._get_rank(output.logits, tokens[:, 1:])

        return ranks, pad_len

    @torch.no_grad
    def decode(self, rank: List[int], pad_len: int):
        generated_ids = torch.full((rank.shape[0], 1), tokenizer.bos_token_id, device=rank.device)
        
        for idx in range(self.context_size):
            output = self.llm(generated_ids)

            logits = output.logits[:, -1, :]  # shape: (n_chunks, vocab)
            logits, sorted_tokens = torch.sort(logits, descending=True, stable=True)

            next_token_id = sorted_tokens.gather(-1, rank[:, idx].unsqueeze(-1))

            generated_ids = torch.cat([generated_ids, next_token_id], dim=1)

        output = generated_ids[:, 1:].flatten()
        return tokenizer.decode(output[:-pad_len], skip_special_tokens=True)

    def evaluate(self, s):
        rank, pad_len = self.encode(s)
        torch.cuda.empty_cache()

        s_hat = self.decode(rank, pad_len)
        assert s_hat == s, f"incorrect (de)-compression \n Expected: {s} \n Got: {s_hat}"

        compressed_s = bz2.compress(s.encode('utf-8'))
        _rank = rank.flatten()
        compressed_s_hat = bz2.compress(_rank.cpu().numpy().tobytes())

        # Get the size of the compressed data
        s_size = len(compressed_s)
        s_hat_size = len(compressed_s_hat)
        # print(s_hat_size, s_size)
        print(f"Compression ratio: {(s_hat_size / s_size)*100:.4f}")

        return _rank, pad_len

llm_zip = LLMCompression(
    llm_name="meta-llama/Llama-3.2-1B",
    context_size=8
)
s = "The quick brown fox jumps over the lazy dog."
rank, pad_len = llm_zip.evaluate(s)

mask torch.Size([2, 8]) torch.Size([2, 8])


AssertionError: incorrect (de)-compression 
 Expected: The quick brown fox jumps over the lazy dog. 
 Got: import connPool
from conn import *
 adult coloring

In [28]:
llm_zip = LLMCompression(
    llm_name="meta-llama/Llama-3.2-1B",
    context_size=8
)
s = "The quick brown fox jumps over the lazy dog."
# s = "The rapid advancement of technology has dramatically reshaped the way humans live, work, and interact with the world. In just a few decades, society has transitioned from relying on traditional forms of communication, such as letters and landline telephones, to an era dominated by smartphones, social media, and artificial intelligence. This transformation has brought numerous benefits, making information more accessible, improving efficiency in various industries, and enhancing global connectivity. However, it has also introduced new challenges, including privacy concerns, the digital divide, and the potential for job displacement due to automation. One of the most significant changes driven by technology is the way people communicate. In the past, communication was often slow and limited to physical mail, face-to-face conversations, or expensive long-distance phone calls. Today, instant messaging, video conferencing, and social media platforms allow individuals to stay connected regardless of geographic location. This has strengthened personal relationships, enabled remote work opportunities, and facilitated the exchange of ideas on a global scale. However, the convenience of digital communication has also led to a decline in face-to-face interactions, raising concerns about its impact on social skills and mental health. Moreover, the rise of misinformation and the spread of fake news through digital platforms pose a significant challenge in today's interconnected world. The ability to share information instantly means that false narratives can gain traction quickly, influencing public opinion and even political outcomes. While technology companies have implemented algorithms and fact-checking mechanisms to combat misinformation, the responsibility ultimately lies with users to critically evaluate the information they consume and share. The integration of artificial intelligence and automation has also transformed various industries, improving productivity and efficiency. In healthcare, AI-powered diagnostic tools assist doctors in identifying diseases more accurately, while robotic surgeries enable precision procedures. In the business sector, automation streamlines supply chains, enhances customer service through chatbots, and improves decision-making with data-driven insights. Despite these advantages, the increasing reliance on technology raises concerns about job displacement, as automation continues to replace human workers in certain roles. This shift necessitates a focus on reskilling and upskilling workers to prepare them for the evolving job market. Education systems must adapt to equip students with the skills needed for the digital age, including proficiency in coding, data analysis, and critical thinking. Additionally, ethical considerations surrounding artificial intelligence must be addressed, ensuring that AI systems are developed and used responsibly. Cybersecurity is another pressing issue in the digital era. With the rise of online transactions, cloud computing, and interconnected devices, cyber threats have become more sophisticated. Data breaches, hacking attempts, and identity theft pose risks to individuals and organizations alike. As a result, cybersecurity measures must continually evolve to protect sensitive information and maintain trust in digital platforms. While technology has undoubtedly improved many aspects of life, it is essential to strike a balance between embracing innovation and addressing its challenges. Responsible use, ethical considerations, and continued education will play a crucial role in shaping a future where technology serves humanity in a positive and sustainable manner."
# s = ":".join(
#     str(x)
#     for x in random.randint(0, 5000, (50,)).tolist()
#     # for x in random.rand(25).tolist()
# )
rank, pad_len = llm_zip.evaluate(s)

The quick brown fox jumps over the lazy dog.
The quick brown fox jumps over the lazy dog.
Compression ratio: 90.1235


In [9]:
llm_zip = LLMCompression(
    llm_name="meta-llama/Llama-3.2-1B",
    context_size=8
)
s = "The quick brown fox jumps over the lazy dog."
# s = "The rapid advancement of technology has dramatically reshaped the way humans live, work, and interact with the world. In just a few decades, society has transitioned from relying on traditional forms of communication, such as letters and landline telephones, to an era dominated by smartphones, social media, and artificial intelligence. This transformation has brought numerous benefits, making information more accessible, improving efficiency in various industries, and enhancing global connectivity. However, it has also introduced new challenges, including privacy concerns, the digital divide, and the potential for job displacement due to automation. One of the most significant changes driven by technology is the way people communicate. In the past, communication was often slow and limited to physical mail, face-to-face conversations, or expensive long-distance phone calls. Today, instant messaging, video conferencing, and social media platforms allow individuals to stay connected regardless of geographic location. This has strengthened personal relationships, enabled remote work opportunities, and facilitated the exchange of ideas on a global scale. However, the convenience of digital communication has also led to a decline in face-to-face interactions, raising concerns about its impact on social skills and mental health. Moreover, the rise of misinformation and the spread of fake news through digital platforms pose a significant challenge in today's interconnected world. The ability to share information instantly means that false narratives can gain traction quickly, influencing public opinion and even political outcomes. While technology companies have implemented algorithms and fact-checking mechanisms to combat misinformation, the responsibility ultimately lies with users to critically evaluate the information they consume and share. The integration of artificial intelligence and automation has also transformed various industries, improving productivity and efficiency. In healthcare, AI-powered diagnostic tools assist doctors in identifying diseases more accurately, while robotic surgeries enable precision procedures. In the business sector, automation streamlines supply chains, enhances customer service through chatbots, and improves decision-making with data-driven insights. Despite these advantages, the increasing reliance on technology raises concerns about job displacement, as automation continues to replace human workers in certain roles. This shift necessitates a focus on reskilling and upskilling workers to prepare them for the evolving job market. Education systems must adapt to equip students with the skills needed for the digital age, including proficiency in coding, data analysis, and critical thinking. Additionally, ethical considerations surrounding artificial intelligence must be addressed, ensuring that AI systems are developed and used responsibly. Cybersecurity is another pressing issue in the digital era. With the rise of online transactions, cloud computing, and interconnected devices, cyber threats have become more sophisticated. Data breaches, hacking attempts, and identity theft pose risks to individuals and organizations alike. As a result, cybersecurity measures must continually evolve to protect sensitive information and maintain trust in digital platforms. While technology has undoubtedly improved many aspects of life, it is essential to strike a balance between embracing innovation and addressing its challenges. Responsible use, ethical considerations, and continued education will play a crucial role in shaping a future where technology serves humanity in a positive and sustainable manner."
# s = ":".join(
#     str(x)
#     for x in random.randint(0, 5000, (50,)).tolist()
#     # for x in random.rand(25).tolist()
# )
rank, pad_len = llm_zip.evaluate(s)

The quick brown fox jumps over the lazy dog.
The quick brown fox jumps over the lazy dog.
Compression ratio is: 90.1235


In [ ]:
s = "The rapid advancement of technology has dramatically reshaped the way humans live, work, and interact with the world. In just a few decades, society has transitioned from relying on traditional forms of communication, such as letters and landline telephones, to an era dominated by smartphones, social media, and artificial intelligence. This transformation has brought numerous benefits, making information more accessible, improving efficiency in various industries, and enhancing global connectivity. However, it has also introduced new challenges, including privacy concerns, the digital divide, and the potential for job displacement due to automation. One of the most significant changes driven by technology is the way people communicate. In the past, communication was often slow and limited to physical mail, face-to-face conversations, or expensive long-distance phone calls. Today, instant messaging, video conferencing, and social media platforms allow individuals to stay connected regardless of geographic location. This has strengthened personal relationships, enabled remote work opportunities, and facilitated the exchange of ideas on a global scale. However, the convenience of digital communication has also led to a decline in face-to-face interactions, raising concerns about its impact on social skills and mental health. Moreover, the rise of misinformation and the spread of fake news through digital platforms pose a significant challenge in today's interconnected world. The ability to share information instantly means that false narratives can gain traction quickly, influencing public opinion and even political outcomes. While technology companies have implemented algorithms and fact-checking mechanisms to combat misinformation, the responsibility ultimately lies with users to critically evaluate the information they consume and share. The integration of artificial intelligence and automation has also transformed various industries, improving productivity and efficiency. In healthcare, AI-powered diagnostic tools assist doctors in identifying diseases more accurately, while robotic surgeries enable precision procedures. In the business sector, automation streamlines supply chains, enhances customer service through chatbots, and improves decision-making with data-driven insights. Despite these advantages, the increasing reliance on technology raises concerns about job displacement, as automation continues to replace human workers in certain roles. This shift necessitates a focus on reskilling and upskilling workers to prepare them for the evolving job market. Education systems must adapt to equip students with the skills needed for the digital age, including proficiency in coding, data analysis, and critical thinking. Additionally, ethical considerations surrounding artificial intelligence must be addressed, ensuring that AI systems are developed and used responsibly. Cybersecurity is another pressing issue in the digital era. With the rise of online transactions, cloud computing, and interconnected devices, cyber threats have become more sophisticated. Data breaches, hacking attempts, and identity theft pose risks to individuals and organizations alike. As a result, cybersecurity measures must continually evolve to protect sensitive information and maintain trust in digital platforms. While technology has undoubtedly improved many aspects of life, it is essential to strike a balance between embracing innovation and addressing its challenges. Responsible use, ethical considerations, and continued education will play a crucial role in shaping a future where technology serves humanity in a positive and sustainable manner."


In [ ]:

compressed_s = bz2.compress(s.encode('utf-8'))

compressed_s_hat = bz2.compress(rank.cpu().numpy().tobytes())

print(len(compressed_s_hat), len(compressed_s))
print(f"Compression ratio is: {(len(compressed_s_hat) / len(compressed_s))*100:.4f}")